# Setup

In [1]:
import os
import pandas as pd

from pprint import pprint

from docx import Document

from sklearn.metrics import accuracy_score

from skllm import MultiLabelZeroShotGPTClassifier
from skllm.config import SKLLMConfig

In [2]:
#See notes in INSTALL.md for how to set this, DO NOT HARD CODE YOUR API KEY HERE, 
# if your repository is public, then someone will steal your API key and make you pay for their shit 
SKLLMConfig.set_openai_key(os.environ.get('OPENAI_API_KEY'))

# Collect a Dataset

In [3]:
# Set the folder path where your Word documents are located
folder_path = 'data'

X = []

# Loop over each file in the folder
for filename in os.listdir(folder_path):
    if filename.endswith('.docx'):
        # Construct the full file path
        file_path = os.path.join(folder_path, filename)
        
        # Open the Word document
        doc = Document(file_path)
        
        # Loop over each paragraph in the document and print its text
        filetext=""
        for para in doc.paragraphs:
            filetext+=para.text
        X.append(filetext)

data = pd.DataFrame(X, columns=['WritingSample'])



# Understand Your Data

In [4]:
data.head()

,WritingSample
0,"Every year, about 1.35 million people are kill..."
1,"In a world driven by technology, some recent a..."
2,The modern world is changing at a concerningly...
3,Technology in today's world continues to take ...
4,Technological improvements are occurring world...


In [5]:
data.describe()

,WritingSample
count,5
unique,5
top,"Every year, about 1.35 million people are kill..."
freq,1


In [6]:
X = data["WritingSample"]

In [7]:
X

0    Every year, about 1.35 million people are kill...
1    In a world driven by technology, some recent a...
2    The modern world is changing at a concerningly...
3    Technology in today's world continues to take ...
4    Technological improvements are occurring world...
Name: WritingSample, dtype: object

# "Develop" a Model aka just use OpenAI's API

In [8]:
# Define candidate labels
candidate_labels = [
    "Great development of main ideas",
    "Average development of main ideas",
    "Poor development of main ideas",
    "Undeveloped",
    "Above average work",
    "Average work",
    "Almost average work",
    "Below average work",
    "Great job of elaborating",
    "Pretty good job of elaborating",
    "Poor job of elaborating",
    "Did not elaborate",
    "Fantastic progression",
    "Progression is sufficient",
    "Progression is unclear",
    "Progression does not make sense",
    "Great use of resources or evidence",
    "Pretty good job using resources or evidence",
    "Inadequate use of resources or evidence",
    "No use of resources or evidence"
]

# Create and fit the classifier
clf = MultiLabelZeroShotGPTClassifier(max_labels=5) 
clf.fit(None, [candidate_labels])

# Define classification prompt (without specifying the writing_sample)
classification_prompt = (
    "Please assess the student's work based on the following criteria:\n\n"
    "Development of Main Ideas:\n"
    "1. Great development of main ideas (1: Poor)\n"
    "2. Average development of main ideas (2: Fair)\n"
    "3. Poor development of main ideas (3: Good)\n"
    "4. Undeveloped (4: Excellent)\n\n"
    "Quality of Work:\n"
    "5. Above average work (1: Poor)\n"
    "6. Average work (2: Fair)\n"
    "7. Almost average work (3: Good)\n"
    "8. Below average work (4: Excellent)\n\n"
    "Elaboration:\n"
    "9. Great job of elaborating (1: Poor)\n"
    "10. Pretty good job of elaborating (2: Fair)\n"
    "11. Poor job of elaborating (3: Good)\n"
    "12. Did not elaborate (4: Excellent)\n\n"
    "Progression:\n"
    "13. Fantastic progression (1: Poor)\n"
    "14. Progression is sufficient (2: Fair)\n"
    "15. Progression is unclear (3: Good)\n"
    "16. Progression does not make sense (4: Excellent)\n\n"
    "Use of Resources or Evidence:\n"
    "17. Great use of resources or evidence (1: Poor)\n"
    "18. Pretty good job using resources or evidence (2: Fair)\n"
    "19. Inadequate use of resources or evidence (3: Good)\n"
    "20. No use of resources or evidence (4: Excellent)\n\n"
    "Please assign an overall score (1 to 4) based on the criteria above. "
    "Please be lenient when assigning scores. Samples were written by low-level high school students."
)

# Extracted 'WritingSample' column
X = data["WritingSample"]

# Iterate through the Writing Samples and classify them
for writing_sample in X:
    # Generate a classification prompt (only criteria)
    criteria_prompt = classification_prompt
    
    # Use the classifier to classify the writing sample
    predicted_labels = clf.predict([criteria_prompt])[0]
    
    # Print the classification result
    print("Predicted Labels:", predicted_labels)
    print()

100%|█████████████████████████████████████████████| 1/1 [00:03<00:00,  3.02s/it]


Predicted Labels: ['Average development of main ideas', 'Average work', 'Pretty good job of elaborating', 'Progression is sufficient', 'Pretty good job using resources or evidence']



100%|█████████████████████████████████████████████| 1/1 [00:02<00:00,  2.44s/it]


Predicted Labels: ['Average development of main ideas', 'Average work', 'Pretty good job of elaborating', 'Progression is sufficient', 'Pretty good job using resources or evidence']



100%|█████████████████████████████████████████████| 1/1 [00:01<00:00,  1.84s/it]


Predicted Labels: ['Average development of main ideas', 'Average work', 'Pretty good job of elaborating', 'Progression is sufficient', 'Pretty good job using resources or evidence']



100%|█████████████████████████████████████████████| 1/1 [00:02<00:00,  2.35s/it]


Predicted Labels: ['Average development of main ideas', 'Average work', 'Pretty good job of elaborating', 'Progression is sufficient', 'Pretty good job using resources or evidence']



100%|█████████████████████████████████████████████| 1/1 [00:02<00:00,  2.18s/it]

Predicted Labels: ['Average development of main ideas', 'Average work', 'Pretty good job of elaborating', 'Progression is sufficient', 'Pretty good job using resources or evidence']



In [9]:
# Predict the labels
labels = clf.predict(X)

100%|█████████████████████████████████████████████| 5/5 [00:10<00:00,  2.05s/it]


In [10]:
# Add labels to the dataset and save
data['ChatGPTLabel'] = labels
data.to_csv('data/classified_tips.csv', index=False)

# Choose a measure of success, Choose an evaluation protocol / evaluate

In [11]:
data[["WritingSample","ChatGPTLabel"]]

,WritingSample,ChatGPTLabel
0,"Every year, about 1.35 million people are kill...","[Great development of main ideas, Great job of..."
1,"In a world driven by technology, some recent a...","[Great development of main ideas, Great job of..."
2,The modern world is changing at a concerningly...,"[Great development of main ideas, Great job of..."
3,Technology in today's world continues to take ...,"[Great development of main ideas, Great job of..."
4,Technological improvements are occurring world...,"[Great development of main ideas, Great job of..."


In [ ]:
accuracy = accuracy_score(data[["HumanLabel"]], data[["ChatGPTLabel"]])
print(accuracy)

# Skipped Steps
* Beat a baseline
* Overfit, regularize and tune
* Communicate with stakeholders
* Ship an inference model
* Monitor and maintain